### 如何評估 RAG 系統的品質?

在 [3_RAG_with_OpenAI.ipynb](3_RAG_with_OpenAI.ipynb) 中，請學員試著評估你修改的 prompt 效果，比起原先講師提供的 prompt，哪個更好

我們是否有一個系統化的指標，可以量化 RAG 系統的好跟壞

1. 人人都會下 prompt，但是誰的 prompt 更好？或是沒差別？
1. 如何選擇 vector store 的 chunking 策略？
1. 哪個 retriever 更好？
1. 要如何持續改善 RAG 系統？下個迭代的改善方向是什麼？

# DevOps 重視持續迭代測試，而 Evaluation 就是 RAG 系統的測試

---

https://docs.ragas.io/en/stable/getstarted/evals/

https://docs.ragas.io/en/stable/getstarted/rag_eval/

---

## 📊 使用 Ragas 評估簡單的 LLM 應用程式

軟體開發流程，做 DevOps 的 CI/CD 迭代測試，來測試產出物的效能與品質。有完整的測試，才能為每個開發變動設立標準。

我們打造一個 RAG System，我們要如何知道這個 RAG System 的效能與品質呢？是看使用者的感覺決定 RAG 的好壞嗎？顯然不是。

### 📌 本指南目標

本指南的目的是展示如何使用 **Ragas** 測試與評估一個簡單的 LLM 應用工作流程。

✅ **Ragas** 是一個用於評估 Retrieval-Augmented Generation 系統表現的 Python 套件，能幫助你衡量回答的準確性、一致性與語意相關性。

✅ **Metrics 評估指標** 是用來量化評估 AI 應用效能的工具。透過這些指標，我們可以判斷整體應用程式以及其內部各個元件在測試資料上的表現好壞。指標提供數據基礎，協助進行比較、最佳化與決策，對於 AI 應用的開發與部署過程至關重要。（例如：faithfulness, answer_relevancy, context_precision 等）

✅ **Evaluation Data / Dataset** 是一組資料集，包含評估指標需要的各項參數，用來表示 Evaluation 的情境或是 test cases。將 RAG 的輸出加上資料集的參數輸入 Metrics 後，Metrics 回傳一個量化的結果，代表 RAG 的效能。

✅ **Evaluation 效能評估** 對於 RAG 系統，以相同的評估資料集做測試，獲得目前 RAG 的量化效能。並依據量化的結果，調整 RAG 系統的元件，例如 embedding, prompt, 使用的 model 等等，持續改善並持續評估。

---

### 🧪 評估場景：RAG

在本示範中，我們將評估一個 RAG System，
目標是確保模型產生的摘要**準確地涵蓋原始文本中的關鍵細節**

---

## 🛠️ 安裝 Ragas

若你想開始使用 **Ragas** 來對 RAG 系統進行評估，只需要使用 `pip` 指令來安裝：

In [1]:
!pip install ragas sacrebleu openai pandas tqdm tenacity ipywidgets

### 🔐 確保你已設定 OpenAI API 金鑰

請將你的 OpenAI API 金鑰設定為環境變數

In [2]:
import os

os.environ["AZURE_OPENAI_API_KEY"]=""
os.environ["AZURE_OPENAI_ENDPOINT"]=""
os.environ["OPENAI_API_VERSION"]="2024-12-01-preview"
os.environ["OPENAI_MODEL"]="gpt-4.1-mini"
#os.environ["OPENAI_MODEL"]="text-embedding-3-large"

if os.getenv("AZURE_OPENAI_API_KEY") is not None:
    print("AZURE_OPENAI_API_KEY is ready")
else:
    print("AZURE_OPENAI_API_KEY environment variable not found")

from openai import AzureOpenAI
openai_client = AzureOpenAI()

AZURE_OPENAI_API_KEY is ready


## 依照上一節的內容，將 RAG 系統開起來

In [3]:
import qdrant_client
from tenacity import retry, wait_random_exponential, stop_after_attempt

@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def get_embedding(text, model="text-embedding-3-large"):
    res = openai_client.embeddings.create(
        model=model,
        input=[text]
    )
    return res.data[0].embedding

def query_docs(query, collection_name="covid-qa-3-large", model="text-embedding-3-large" , top_k=5):
    query_vect = get_embedding(query, model)
    results = client.query_points(
        collection_name=collection_name,
        query=query_vect,
        limit=5,
        with_payload=True,
        using="title"
    )
    payloads = [point.payload["answer"] for point in results.points]
    return payloads

@retry(
    wait=wait_random_exponential(min=1, max=60),  # backoff 等待時間
    stop=stop_after_attempt(6),  # 最多重試 6 次
)
def generate_answer(query, docs, model="gpt-4.1-mini"):
    context = "\n\n".join(docs)
    prompt = f"""根據以下內容回答問題：

1. 請用繁體中文回答
2. 依照內容產生回答
3. 附上內容原文作為依據，原文保留內容的原始語言
4.. 如果內容不包含就回答你不知道

內容：
{context}

問題：
{query}
"""

    res = openai_client.chat.completions.create(
        model=model,  # 或你的 Azure 模型名稱
        messages=[
            {"role": "system", "content": "你是一個 helpful AI 助理"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return res.choices[0].message.content.strip()

client = qdrant_client.QdrantClient(
    host="qdrant",
    prefer_grpc=True,
)

### 🔄 RAG 查詢

In [4]:
query = "COVID 的全名是什麼"
docs = query_docs(
    query=query,
    collection_name="covid-qa-3-large",
    model="text-embedding-3-large")

answer = generate_answer(
    query=query, 
    docs=docs, 
    model="gpt-4.1-mini")

print("\n🧠 回答：")
print(answer)


🧠 回答：
COVID 的全名是「Coronavirus Disease 2019」，簡稱 COVID-19。

依據原文：
"WHO announced “COVID-19” as the name of this new disease on 11 February 2020, following guidelines previously developed with the World Organisation for Animal Health (OIE) and the Food and Agriculture Organization of the United Nations (FAO)."


### 📐 Ragas 提供的評估方法

Ragas 提供多種評估 LLM 應用效能的方法，我們稱這些為 **評估指標（metrics）**。

每個評估指標都需要一組預先定義的資料點（data points），
這些資料會用來計算「效能分數」，幫助你理解模型的表現。

---

### ✅ 使用非 LLM 評估指標（Non-LLM Metric）

以下是一個使用 **BLEU 分數**（`BleuScore`）的簡單評估範例，
這是一種不依賴大型語言模型的傳統語言相似度衡量方式，常用於摘要或翻譯任務。

https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/traditional/

In [5]:
from ragas import SingleTurnSample
from ragas.metrics import BleuScore

test_data = {
    "user_input": "summarise given text\nThe company reported an 8% rise in Q3 2024, driven by strong performance in the Asian market. Sales in this region have significantly contributed to the overall growth. Analysts attribute this success to strategic marketing and product localization. The positive trend in the Asian market is expected to continue into the next quarter.",
    "response": "The company experienced an 8% increase in Q3 2024, largely due to effective marketing strategies and product adaptation, with expectations of continued growth in the coming quarter.",
    "reference": "The company reported an 8% growth in Q3 2024, primarily driven by strong sales in the Asian market, attributed to strategic marketing and localized products, with continued growth anticipated in the next quarter."
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.13718598426177148

### 🔄 改用 RAG 的輸入與輸出

In [6]:
test_data = {
    "user_input": "What is the full name of COVID?",
    "response": 'The full name of COVID is "Coronavirus Disease 2019"',
    "reference": "WHO announced “COVID-19” as the name of this new disease on 11 February 2020, following guidelines previously developed with the World Organisation for Animal Health (OIE) and the Food Agriculture Organization of the United Nations (FAO)."
}
metric = BleuScore()
test_data = SingleTurnSample(**test_data)
metric.single_turn_score(test_data)

0.004475925962507959

## 📋 我們在這裡使用了：

* 一筆測試資料（**test sample**），包含以下欄位：

  * `user_input`：使用者輸入
  * `response`：由 LLM 模型生成的輸出
  * `reference`：理想或預期的輸出結果（作為評估依據）

* 一個非 LLM 型的評估指標：`BleuScore`

---

## ⚠️ 此方法的兩大限制：

### 1️⃣ 需要大量準備工作

評估這類應用時，**必須為每個輸入預先準備對應的理想輸出（reference）**。
這種方式不僅耗時，還很難在大規模應用中維護。

### 2️⃣ 評分結果可能不準確

即使實際輸出（`response`）與預期輸出（`reference`）非常相似，
**`BleuScore` 所計算出的分數仍可能偏低**。

這是傳統非語言模型指標（例如 `Bleu`, `ROUGE`, `METEOR` 等）的既有問題：
它們**只依據詞彙重合率進行評分**，而無法理解語意上的相似性。

---

### ✅ 建議使用

為了解決上述問題，建議使用 **Ragas 提供的 LLM 驅動評估指標**（例如：`faithfulness`, `answer_relevancy`, `context_precision` 等），
這些指標會使用語言模型來比較輸出與參考答案的語意一致性，更能反映實際表現。

## 🤖 使用 LLM 為基礎的評估指標進行評估

相較於傳統的非 LLM 指標（如 BLEU 或 ROUGE），**LLM-based metrics** 能理解語意與上下文，
可更準確地評估大型語言模型（LLM）所生成的回應品質。

LLM-based 指標（例如 `faithfulness`, `answer_relevancy`, `context_precision` 等）能夠深入理解語意、上下文與問題間的關係，提供更準確的評估結果，適合用於 RAG 系統。

---

## 載入資料集

### 📁 資料集簡介：`covidqa`

這是一組基於 **COVID-19 文獻資料** 建立的問答資料集，目的是幫助開發者訓練自然語言理解（NLU）與問答系統，以回應與疫情相關的問題。

---

### 📄 檔案：`community.csv` 是什麼？

這個 `community.csv` 是該資料集中的一個檔案，根據名稱與用途，它大致包含從 COVID-19 社群討論（例如研究社群、論壇、問答平台等）中擷取出來的 **問答對（Question-Answer Pairs）**。

---

### 🔍 用途

* 用於訓練問答模型（如：BERT、GPT + 向量資料庫）。
* 適合建立 Retrieval-Augmented Generation (RAG) 系統。
* 可搭配向量嵌入技術（如 OpenAI Embedding + Qdrant）建立問答應用。

# 收集評估資料（Evaluation Data）

為了收集評估資料，我們首先需要準備一組查詢（queries），用來對 RAG 系統進行測試。

## 步驟說明

1. **準備查詢集**  
   準備一組查詢，這些查詢將被送入 RAG 系統。

2. **執行查詢**  
   將這些查詢透過 RAG 系統執行，並收集以下資料：
   - 系統回應（response）
   - 為每個查詢所擷取的上下文內容（retrieved_contexts）

3. **（選用）準備標準答案**  
   可以選擇為每個查詢準備一組黃金標準答案（golden answers），用來進一步評估系統的表現。

---

### 🔍 原始資料集取樣，產生 Evaluation 資料集

* 使用更完整的資料及進行測試會花更多時間
* 也會花更多成本在 OpenAI API

為了節省時間，這邊只採用原始資料集中的 30 個 sample

---

### 🔍 現實中如何產生 Evaluation 資料集

1. 請 COVID 專家，產生專業的 COVID 問答，作為資料集。缺點是專家很貴，很花時間
2. 使用 LLM + testset generator 產生全新的 COVID 問答資料集。由於 LLM 普及，成本變得很低
3. 在剛開始做 RAG 前，保留一部分原始資料集，不要寫入 Qdrant，作為 Evaluation 資料集

---

### 本 workshop 的做法

由於原始資料集就有提供 question + answer，這邊直接使用寫入 RAG database 的原始資料集當作評估資料集

### 這樣會有什麼問題嗎？

如果 RAG 是一個測試的學生，這邊相當於讓學生讀一本資料，考試的時候的資料又是同一本書。

聽起來怪怪的，但這邊先不展開

In [7]:
import pandas as pd

community_df = pd.read_csv('/home/jovyan/community.csv')
eval_samples=community_df.sample(n=10)
eval_samples

,question_id,title,question,answer_id,answer,answer_type,wrong_answer,wrong_answer_type,url,source
504,52322,Why have some governments banned outbound inte...,Why have some governments banned outbound inte...,52326,"Another possibility, ignoring fears of infecti...",Reasonable,"As you rightly mentioned in your question, arg...",Random,politics.stackexchange.com,general
306,145815,Chair Offers COVID-19 Compensation... But Asks...,The chair of my department has very generously...,145818,There are really two parts to the Chair's ques...,Reasonable,I actually am in a similar situation from time...,Bad Answers,academia.stackexchange.com,expert
146,21571,ACE2 Inhibition as therapeutic candidate?,"Disclaimer: I am a scientist, but this is not ...",21658,The problem appears to be that the SARS-CoV-2 ...,Reasonable,Just as aluminium compounds such as:\n\n\nAlum...,Random,health.stackexchange.com,biomedical
114,22937,Would the human seasonal coronaviruses be just...,Presuming that we were able to find a person w...,22938,The seasonal coronaviruses attach exclusively ...,Accepted,"If I understand your question right, the probl...",Random,health.stackexchange.com,biomedical
395,123277,Should I remove Adult Dependent from my 2019 t...,On my 2019 Tax Return I claimed my 28 year old...,123299,"You choosing to remove her as your dependent, ...",Reasonable,\n Chances are everyone who reads this StackE...,Random,money.stackexchange.com,general
344,39689,How to get 5-year-old to consistently cover mo...,My 5-year-old son started developing a dry cou...,39690,Getting someone into the habit of doing anythi...,Accepted,Don't talk - just act:\n\nEvery time the child...,Bad Answers,parenting.stackexchange.com,general
286,147397,Letting grant funder know that funds no longer...,I am a English PhD student (focusing on litera...,147398,I would ask the representative. Procedures dif...,Accepted,"I can provide some information about the US, a...",Random,academia.stackexchange.com,expert
292,145102,How to detect cheating when students take onli...,I am currently teaching a basic R programming ...,145105,"I also teach a data and programming course, th...",Reasonable,One thing not mentioned yet is that publisher ...,Random,academia.stackexchange.com,expert
398,123381,Coronavirus crisis: why Microsoft & Amazon mor...,just checking some stock prices:\n\n\nGoogle: ...,123382,"There may not be one definitive reason, but I'...",Reasonable,\n What factors should be considered? \n\n\n\...,Random,money.stackexchange.com,general
436,51000,Does the World Health Organization have the po...,"In the last couple of days, WHO officials have...",51009,\n WHO officials have criticised the UK gover...,Accepted,In Algeria currently this problem exists in a ...,Random,politics.stackexchange.com,general


### 直接使用寫入 RAG database 的原始資料集當作評估資料集

根據後面 evaluate metrics 之標所需要的欄位，產生 user_input, retrieved_contexts, response, reference 欄位的資料，其中

1. user_input 是輸入 RAG 的 query 問題
2. retrieved_contexts 是 RAG 搜尋 qdrant 的 contexts
3. response 是 RAG 回覆 input 的回答
4. reference 是原始資料集的原始答案，這裡我們把它當作是標準答案。也就是評分的標準之一

In [8]:
from tqdm import tqdm

dataset = []

for index, row in tqdm(eval_samples.iterrows(), total=len(eval_samples), desc="Building eval dataset"):
    query = row["title"]
    relevant_docs = query_docs(
        query=query,
        collection_name="covid-qa-3-large",
        model="text-embedding-3-large")
    response = generate_answer(
        query=query,
        docs=relevant_docs,
        model="gpt-4.1-mini")
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": relevant_docs,
            "response": response,
            "reference": row["answer"]
        }
    )

Building eval dataset: 100%|██████████| 10/10 [01:08<00:00,  6.82s/it]


### 檢視產生的評估資料集

In [9]:
dataset[0]

{'user_input': 'What happens to viruses that causes them to die?',
 'retrieved_contexts': ['https://www.sciencefocus.com/nature/do-viruses-die/:\n\n\n  Strictly speaking, viruses can’t die, for the simple reason that they aren’t alive in the first place. Although they contain genetic instructions in the form of DNA (or the related molecule, RNA), viruses can’t thrive independently. Instead, they must invade a host organism and hijack its genetic instructions.\n\n\nhttps://www.sciencealert.com/how-long-does-coronavirus-last-on-surfaces:\n\n\n  Just hanging about in the atmosphere, the effect of factors such as UV light and heat causes the mix of RNA, fatty membrane, and protein making up the [viruses] to steadily break down in a few hours [or less/more].\n\n\nhttps://www.statnews.com/2020/03/19/coronavirus-survives-on-surfaces-how-to-protect-yourself/ (mirror):\n\n\n  Viruses covered in “envelopes” have the most trouble surviving outside a living cell. On surfaces, the surrounding light

In [10]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

# 評估（Evaluate）

我們已成功收集評估資料，接下來可以使用一組常見的 RAG 評估指標（evaluation metrics）對我們的 RAG 系統進行評估。

## 評估步驟

1. **使用收集到的資料集**  
   基於先前收集的查詢、系統回應及擷取的上下文，進行模型效能評估。

2. **選擇評估指標**  
   使用常見的 RAG 評估指標，例如：
   - 回答正確性（Answer Correctness）
   - 上下文相關性（Context Relevance）
   - 回應完整性（Answer Completeness）

3. **選擇評估模型**  
   你可以選擇任意一個 LLM 作為評估器（Evaluator LLM）來執行這些評估任務。

這個 Evaluator LLM 可以想像是打分數的裁判，所以不一定要使用跟 RAG 相同的模型。

由於我們希望評估 answer_relevancy, answer_correctness and answer_similarity 等指標，這些指標與 embedding 有關，所以在設定 Evaluator 時也提供 AzureOpenAIEmbeddings，也是裁判的一部分。

https://docs.ragas.io/en/v0.1.21/howtos/customisations/azure-openai.html

In [11]:
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate

azure_model = AzureChatOpenAI(
    model="gpt-4.1-mini",
    validate_base_url=False,
)

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
azure_embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
)

https://docs.ragas.io/en/stable/concepts/metrics/overview/#different-types-of-metrics

# 評估（Evaluate）

在成功收集完評估資料後，我們就可以使用一組常見的 RAG 評估指標，針對收集到的資料集來評估 RAG 系統的效能。

你可以選擇任何一個模型作為評估用的 LLM（Evaluator LLM）來進行評分與比較。

---

# 評估指標概觀（Overview of Metrics）

「指標（Metric）」是用來量化評估 AI 應用效能的工具。透過這些指標，我們可以判斷整體應用程式以及其內部各個元件在測試資料上的表現好壞。指標提供數據基礎，協助進行比較、最佳化與決策，對於 AI 應用的開發與部署過程至關重要。

## 指標的重要性

- **元件選擇（Component Selection）**  
  可使用指標來比較 AI 應用中的不同元件（如 LLM、Retriever、Agent 組態等）在自有資料上的表現，從多個選項中選出最佳組合。

- **錯誤診斷與除錯（Error Diagnosis and Debugging）**  
  指標能協助找出應用中表現不佳或發生錯誤的元件，有助於釐清問題並加以改進。

- **持續監控與維運（Continuous Monitoring and Maintenance）**  
  指標可用於長期追蹤 AI 應用的表現，幫助偵測如資料漂移、模型效能下降或用戶需求變化等問題，並及時調整應對。

---

# 常見 RAG 評估指標說明

## 1. LLM Context Recall（LLM 上下文召回率）
- **說明**：衡量 RAG 系統回應中是否使用了檢索到的上下文資訊。
- **目標**：確保模型的回答有依據地參考了相關的背景資料。

## 2. Faithfulness（忠實度）
- **說明**：評估回應是否忠實反映檢索到的內容。
- **目標**：避免 RAG 系統編造（hallucinate）資訊，確保回答與來源資料一致。

## 3. Factual Correctness（事實正確性）
- **說明**：判斷 RAG 系統回應的事實是否正確，不僅限於上下文來源，也包括一般常識與外部知識。
- **目標**：減少錯誤或誤導性的回答。

## 4. Noise Sensitivity（雜訊敏感度）
- **說明**：評估模型在面對含有雜訊或不相關資訊的上下文時，是否仍能做出正確判斷。
- **目標**：強化模型的抗雜訊能力，提升實用性。

## 5. Response Relevancy（回應相關性）
- **說明**：評估 RAG 系統回應是否與使用者查詢高度相關。
- **目標**：確保回答內容精準聚焦於問題本身。

## 6. Context Entity Recall（上下文實體召回率）
- **說明**：檢查模型是否成功提取上下文中與查詢相關的重要實體（例如人名、地名、產品名等）。
- **目標**：提升資訊擷取的完整性與語意理解。

---

這些指標可以單獨或組合使用，幫助開發者全面了解 RAG 系統在真實應用中的表現，進而針對特定弱點進行優化。

In [15]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, NoiseSensitivity, ResponseRelevancy, ContextEntityRecall

result = evaluate(
    evaluation_dataset, 
    metrics=[
        # 需要執行 metrics X EvaluationDataset 數量的次數，會花比較久時間。
        # 可能會有 TimeoutError() https://github.com/explodinggradients/ragas/issues/1280
        LLMContextRecall(), 
        Faithfulness(), 
        FactualCorrectness(),
        NoiseSensitivity(),
        ResponseRelevancy(),
        ContextEntityRecall()
    ],
    llm=azure_model, 
    embeddings=azure_embeddings
)


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

Exception raised in Job[9]: TimeoutError()
Exception raised in Job[21]: TimeoutError()
Exception raised in Job[45]: TimeoutError()


### 獲得每個 Metrics 指標個一個的量化分數

結果依據各個指標不同，需要個別去查文件說明，例如 Faithfulness

https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/faithfulness/

**Faithfulness（真實性）** 是用來評估 RAG（Retrieval-Augmented Generation）系統回應是否忠實於檢索到的文件內容的指標。

- 它會檢查模型生成的答案是否有「捏造（hallucination）」資訊。
- 具體來說，faithfulness 透過 LLM（如 GPT）判斷生成的回答中，哪些句子與原始上下文資料不一致。
- 分數介於 0 到 1，越接近 1 表示回答越忠實於來源文件
- 適用於檢查生成答案是否可信且有依據。

https://docs.ragas.io/en/latest/concepts/metrics/available_metrics/factual_correctness/

**Factual Correctness（事實正確性）** 是 RAG 系統中用來評估回答是否正確反映問題事實的指標。

- 不同於 `faithfulness` 專注於與文件的一致性，`factual_correctness` 更關注回答本身是否合理且符合問題所需的真實資訊。
- 它會使用 LLM 檢查回答是否答對了問題，並給出 0 到 1 的分數，1 表示完全正確。
- 此指標可搭配其他指標（如 contextual precision）使用，提供更全面的回應品質評估。


In [16]:
result

{'context_recall': 1.0000, 'faithfulness': 0.9502, 'factual_correctness(mode=f1)': 0.6850, 'noise_sensitivity(mode=relevant)': 0.2083, 'answer_relevancy': 0.5224, 'context_entity_recall': 0.7958}

### 結果中可以查詢每一筆評估資料獲得的分數

In [17]:
df = result.to_pandas()
df.head()

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1),noise_sensitivity(mode=relevant),answer_relevancy,context_entity_recall
0,What happens to viruses that causes them to die?,[https://www.sciencefocus.com/nature/do-viruse...,根據提供的內容，病毒本身並不算是活的生物，因此嚴格來說病毒不會「死亡」。病毒需要侵入宿主細胞...,https://www.sciencefocus.com/nature/do-viruses...,1.0,1.000000,0.68,0.521739,0.674641,0.846154
1,"Do I need passport for US domestic flights, du...","[Update March 27, 2020:\n\nTo reduce the need ...",根據內容，您在2021年10月1日之後搭乘美國國內航班時，需要出示符合REAL ID標準的駕...,"Update March 27, 2020:\n\nTo reduce the need f...",1.0,1.000000,0.83,NaN,0.696770,0.923077
2,Are all emerging viral diseases of the past 10...,"[To my knowledge, yes. A partial list of recen...",根據內容，並非所有過去100年出現的新興病毒疾病都是人畜共通傳染病（zoonoses），但多...,"To my knowledge, yes. A partial list of recent...",1.0,0.818182,0.51,0.000000,0.000000,0.942857
3,Can I sue my employer if they terminate my emp...,[People are laid off all the time when sales a...,根據提供的內容，您通常無法因為在新冠病毒疫情期間被裁員而起訴您的雇主。法律上並沒有「工作權利...,People are laid off all the time when sales ar...,1.0,0.950000,0.94,NaN,0.902926,1.000000
4,COVID-19: some EU countries closed borders for...,"[Yes, EU citizens are foreigners. The Czech re...",根據內容，歐盟（EU）公民在捷克被視為外國人（foreigners）。捷克的規定只允許捷克公...,"Yes, EU citizens are foreigners. The Czech reg...",1.0,0.933333,0.76,0.000000,0.630447,0.818182


---
# 小結: Embedding

1. 針對 RAG 系統做 Evaluation
1. 使用 Ragas 提供的評估方法與 Metrics
1. 收集評估資料（Evaluation Data）。由於沒有準備評估資料集，我們使用 OpenAI chat 根據 question 產生一組 answer 作為評估資料集 
1. 選擇我們需要的指標，每個指標需要的評估資料集的欄位有所不同，需要滿足欄位
1. 進行評估（Evaluate），所需時間隨指標數量與評估資料集數量成長

---

# 👋👋👋自己動手做看看👋👋👋

嘗試使用不同的 collection_name="covid-qa-3-small", 運作 RAG 後，產生新的評估資料集，並進行評估

---

### 提示

1. 裁判 Evaluator model 需要更換嗎？換裁判算出來的分數公平嗎？
1. 如果 collection 與 model 使用不同的 embedding model，結果跑得出來嗎？`collection_name="covid-qa-3-large" model="text-embedding-3-large"`


In [ ]:
community_df = pd.read_csv('/home/jovyan/community.csv')
eval_samples=community_df.sample(n=10)
eval_samples

small_dataset = []

for index, row in tqdm(eval_samples.iterrows(), total=len(eval_samples), desc="Building eval dataset"):
    # ...
        collection_name="covid-qa-3-small", # 換成 small collection
        model="text-embedding-3-small") # 換成 small embedding model
    # ...

small_evaluation_dataset = []

result = evaluate(
    small_evaluation_dataset, 
    metrics=[
        Faithfulness(), 
        FactualCorrectness(),
    ],
    llm=azure_model, # 裁判 Evaluator model 需要更換嗎？換裁判算出來的分數公平嗎？
    embeddings=azure_embeddings # embedding model 需要更換嗎？
)
